In [19]:
import numpy as np 
import pandas as pd
#from pandas_profiling import ProfileReport
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
#from fancyimpute import KNN



In [20]:
churn = pd.read_csv('../data/churn.csv')

In [21]:
churn.head(20)

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,city,last_trip_date,phone,signup_date,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct
0,3.67,5.0,4.7,1.10,King's Landing,2014-06-17,iPhone,2014-01-25,15.4,4,True,46.2
1,8.26,5.0,5.0,1.00,Astapor,2014-05-05,Android,2014-01-29,0.0,0,False,50.0
2,0.77,5.0,4.3,1.00,Astapor,2014-01-07,iPhone,2014-01-06,0.0,3,False,100.0
3,2.36,4.9,4.6,1.14,King's Landing,2014-06-29,iPhone,2014-01-10,20.0,9,True,80.0
4,3.13,4.9,4.4,1.19,Winterfell,2014-03-15,Android,2014-01-27,11.8,14,False,82.4
5,10.56,5.0,3.5,1.00,Winterfell,2014-06-06,iPhone,2014-01-09,0.0,2,True,100.0
6,3.95,4.0,NaN,1.00,Astapor,2014-01-25,Android,2014-01-24,0.0,1,False,100.0
7,2.04,5.0,5.0,1.00,Winterfell,2014-01-29,iPhone,2014-01-28,0.0,2,False,100.0
8,4.36,5.0,4.5,1.00,Winterfell,2014-02-01,Android,2014-01-21,0.0,2,False,100.0
9,2.37,5.0,NaN,1.00,Winterfell,2014-01-05,Android,2014-01-03,0.0,1,False,0.0


In [22]:
churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   avg_dist                50000 non-null  float64
 1   avg_rating_by_driver    49799 non-null  float64
 2   avg_rating_of_driver    41878 non-null  float64
 3   avg_surge               50000 non-null  float64
 4   city                    50000 non-null  object 
 5   last_trip_date          50000 non-null  object 
 6   phone                   49604 non-null  object 
 7   signup_date             50000 non-null  object 
 8   surge_pct               50000 non-null  float64
 9   trips_in_first_30_days  50000 non-null  int64  
 10  luxury_car_user         50000 non-null  bool   
 11  weekday_pct             50000 non-null  float64
dtypes: bool(1), float64(6), int64(1), object(4)
memory usage: 4.2+ MB


In [23]:
#Simple mean impute for ratings columns
imp = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imp.fit(churn[['avg_rating_by_driver','avg_rating_of_driver']])
churn_imp = imp.transform(churn[['avg_rating_by_driver','avg_rating_of_driver']])
pd.DataFrame(churn_imp)
churn[['avg_rating_by_driver','avg_rating_of_driver']]=churn_imp
churn

#encode non-null values: https://towardsdatascience.com/preprocessing-encode-and-knn-impute-all-categorical-features-fast-b05f50b4dfaa 
encoder = OrdinalEncoder()
#imputer = KNN()
cat_cols = ['phone']

def encode(data):
    '''function to encode non-null data and replace it in the original data'''
    #retains only non-null values
    nonulls = np.array(data.dropna())
    #reshapes the data for encoding
    impute_reshape = nonulls.reshape(-1,1)
    #encode date
    impute_ordinal = encoder.fit_transform(impute_reshape)
    #Assign back encoded values to non-null values
    data.loc[data.notnull()] = np.squeeze(impute_ordinal)
    return data


for columns in cat_cols:
    encode(churn[columns])

def churn_(df, column):
    dates = df[column]
    cutoff_date = '2014-06-01'
    array = []
    for date in dates:
        if date > cutoff_date:
            array.append(1)
        else:
            array.append(0)
    df['active'] = array
    return df


churn = churn_(churn, 'last_trip_date')


#KNN impute for phone column
imputer = KNNImputer(n_neighbors=5, weights="uniform")
churn_imp2=imputer.fit_transform(churn[['avg_rating_by_driver','avg_rating_of_driver','phone']])
churn[['avg_rating_by_driver','avg_rating_of_driver','phone']]=churn_imp2


# creating instance of one-hot-encoder 
enc = OneHotEncoder(handle_unknown='ignore')
enc_df = pd.DataFrame(enc.fit_transform(churn[['city']]).toarray())
churn = churn.join(enc_df)
churn = churn.drop(['city'],axis =1)

#create feature with difference between signup and last trip
churn['last_trip_date_d'] = pd.to_datetime(churn['last_trip_date'])
churn['signup_date_d'] = pd.to_datetime(churn['signup_date'])
churn['date_delta'] = churn['last_trip_date_d']-churn['signup_date_d']



In [24]:

churn = churn.rename(columns = {0:'city_0', 1:'city_1',2:'city_2'},errors = 'raise')


In [25]:
churn

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,last_trip_date,phone,signup_date,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct,active,city_0,city_1,city_2,last_trip_date_d,signup_date_d,date_delta
0,3.67,5.0,4.700000,1.10,2014-06-17,1.0,2014-01-25,15.4,4,True,46.2,1,0.0,1.0,0.0,2014-06-17,2014-01-25,143 days
1,8.26,5.0,5.000000,1.00,2014-05-05,0.0,2014-01-29,0.0,0,False,50.0,0,1.0,0.0,0.0,2014-05-05,2014-01-29,96 days
2,0.77,5.0,4.300000,1.00,2014-01-07,1.0,2014-01-06,0.0,3,False,100.0,0,1.0,0.0,0.0,2014-01-07,2014-01-06,1 days
3,2.36,4.9,4.600000,1.14,2014-06-29,1.0,2014-01-10,20.0,9,True,80.0,1,0.0,1.0,0.0,2014-06-29,2014-01-10,170 days
4,3.13,4.9,4.400000,1.19,2014-03-15,0.0,2014-01-27,11.8,14,False,82.4,0,0.0,0.0,1.0,2014-03-15,2014-01-27,47 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,5.63,4.2,5.000000,1.00,2014-06-05,1.0,2014-01-25,0.0,0,False,100.0,1,0.0,1.0,0.0,2014-06-05,2014-01-25,131 days
49996,0.00,4.0,4.601559,1.00,2014-01-25,1.0,2014-01-24,0.0,1,False,0.0,0,1.0,0.0,0.0,2014-01-25,2014-01-24,1 days
49997,3.86,5.0,5.000000,1.00,2014-05-22,0.0,2014-01-31,0.0,0,True,100.0,0,0.0,0.0,1.0,2014-05-22,2014-01-31,111 days
49998,4.58,3.5,3.000000,1.00,2014-01-15,1.0,2014-01-14,0.0,2,False,100.0,0,1.0,0.0,0.0,2014-01-15,2014-01-14,1 days


In [26]:
def date_to_month(df, column_name):
    df[column_name] = pd.to_datetime(df[column_name])
    df['month '+ column_name] = df[column_name].dt.month
    return df

date_to_month(churn, 'signup_date')
churn = churn.drop(['signup_date'],axis =1)
date_to_month(churn, 'last_trip_date')
churn = churn.drop(['last_trip_date'], axis = 1)
churn = churn.drop(['last_trip_date_d','signup_date_d'], axis = 1)



In [27]:
churn

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,phone,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct,active,city_0,city_1,city_2,date_delta,month signup_date,month last_trip_date
0,3.67,5.0,4.700000,1.10,1.0,15.4,4,True,46.2,1,0.0,1.0,0.0,143 days,1,6
1,8.26,5.0,5.000000,1.00,0.0,0.0,0,False,50.0,0,1.0,0.0,0.0,96 days,1,5
2,0.77,5.0,4.300000,1.00,1.0,0.0,3,False,100.0,0,1.0,0.0,0.0,1 days,1,1
3,2.36,4.9,4.600000,1.14,1.0,20.0,9,True,80.0,1,0.0,1.0,0.0,170 days,1,6
4,3.13,4.9,4.400000,1.19,0.0,11.8,14,False,82.4,0,0.0,0.0,1.0,47 days,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,5.63,4.2,5.000000,1.00,1.0,0.0,0,False,100.0,1,0.0,1.0,0.0,131 days,1,6
49996,0.00,4.0,4.601559,1.00,1.0,0.0,1,False,0.0,0,1.0,0.0,0.0,1 days,1,1
49997,3.86,5.0,5.000000,1.00,0.0,0.0,0,True,100.0,0,0.0,0.0,1.0,111 days,1,5
49998,4.58,3.5,3.000000,1.00,1.0,0.0,2,False,100.0,0,1.0,0.0,0.0,1 days,1,1


In [32]:
churn['date_delta'] = churn['date_delta'].dt.days

In [35]:
churn

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,phone,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct,active,city_0,city_1,city_2,date_delta,month signup_date,month last_trip_date
0,3.67,5.0,4.700000,1.10,1.0,15.4,4,True,46.2,1,0.0,1.0,0.0,143,1,6
1,8.26,5.0,5.000000,1.00,0.0,0.0,0,False,50.0,0,1.0,0.0,0.0,96,1,5
2,0.77,5.0,4.300000,1.00,1.0,0.0,3,False,100.0,0,1.0,0.0,0.0,1,1,1
3,2.36,4.9,4.600000,1.14,1.0,20.0,9,True,80.0,1,0.0,1.0,0.0,170,1,6
4,3.13,4.9,4.400000,1.19,0.0,11.8,14,False,82.4,0,0.0,0.0,1.0,47,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,5.63,4.2,5.000000,1.00,1.0,0.0,0,False,100.0,1,0.0,1.0,0.0,131,1,6
49996,0.00,4.0,4.601559,1.00,1.0,0.0,1,False,0.0,0,1.0,0.0,0.0,1,1,1
49997,3.86,5.0,5.000000,1.00,0.0,0.0,0,True,100.0,0,0.0,0.0,1.0,111,1,5
49998,4.58,3.5,3.000000,1.00,1.0,0.0,2,False,100.0,0,1.0,0.0,0.0,1,1,1
